## 多线程爬虫 

In [ ]:
# 传统运行代码
import time

def coding():
    for x in range(3):
        print('coding:',x)
        time.sleep(1)

def drawing():
    for x in range(3):
        print('drawing:',x)
        time.sleep(1)
        
if __name__ == '__main__':
    coding()
    drawing()

In [ ]:
# 多线程方式
import threading
import time

def coding():
    for x in range(3):
        print('coding:',threading.current_thread())
        time.sleep(1)

def drawing():
    for x in range(3):
        print('drawing:',threading.current_thread())
        time.sleep(1)
        
def main():
    t1 = threading.Thread(target=coding)
    t2 = threading.Thread(target=drawing)
    
    t1.start()
    t2.start()
    
    print(threading.enumerate())
        
if __name__ == '__main__':
    main()

In [ ]:
## 继承threading.Thread类

In [ ]:
import threading
import time

class Coding(threading.Thread):
    def run(self):
        for x in range(3):
            print('%s正在Coding...' % threading.current_thread())
            time.sleep(1)

class Drawing(threading.Thread):
    def run(self):
        for x in range(3):
            print('%s正在Drawing...' % threading.current_thread())
            time.sleep(1)
            
def main():
    t1 = Coding()
    t2 =Drawing()
    t1.start()
    t2.start()
    
if __name__ == '__main__':
    main()

## 多线程共享全局变量的问题

In [ ]:
import threading

value = 0

# 在需要修改全局变量的地方加锁
g_lock  = threading.Lock()

def add_val():
    global value
    g_lock.acquire()
    for x in range(1000000):
        value += 1
    g_lock.release()
    print('Value: ', value)
    
def main():
    for x in range(2):
        t = threading.Thread(target=add_val)
        t.start()
    
if __name__ == '__main__':
    main()



## Lock版本生产者和消费者模式

In [ ]:
import threading
import random
import time

gMoney = 1000
gLock = threading.Lock()

gTotalTimes = 10
gTimes = 0

class Producer(threading.Thread):
    def run(self):
        global gMoney
        global gTimes
        while True:
            money = random.randint(100, 1000)
            gLock.acquire()
            if gTimes >= gTotalTimes:
                gLock.release()
                break
            gMoney += money
            gTimes += 1
            print('%s生产者第%d次生产了%d元， 剩余%d元' % (threading.current_thread(),gTimes, money, gMoney))
            
            gLock.release()
            time.sleep(0.5)
        
    
class Consumer(threading.Thread):
    def run(self):
        global gMoney 
        while True:
            money = random.randint(100, 1000)
            gLock.acquire()
            if gMoney >= money:
                gMoney -= money
                print('%s消费者第消费了%d元，剩余%d元' % (threading.current_thread(), money, gMoney))
            else:
                if gTimes >= gTotalTimes:
                    gLock.release()
                    break
                print('余额不足，需要消费%d元，仅剩%d元' % (money, gMoney))
            gLock.release()
            time.sleep(0.5)
    
def main():
    for x in range(3):
        t1 = Consumer(name='消费者%s' % x)
        t1.start()
        
    for x in range(5):
        t2 = Producer(name='生产者%s' % x)
        t2.start()
        
if __name__ == '__main__':
    main()

## Condition版本的生产者和消费者模式

In [ ]:
import threading
import random
import time

gMoney = 1000
gCondition = threading.Condition()

gTotalTimes = 10
gTimes = 0

class Producer(threading.Thread):
    def run(self):
        global gMoney
        global gTimes
        while True:
            money = random.randint(100, 1000)
            gCondition.acquire()
            if gTimes >= gTotalTimes:
                gCondition.release()
                break
            gMoney += money
            gTimes += 1
            print('生产者[%s]第%d次生产了%d元， 剩余%d元' % (threading.current_thread(),gTimes, money, gMoney))
            gCondition.notify_all()
            gCondition.release()
            time.sleep(0.5)
        
    
class Consumer(threading.Thread):
    def run(self):
        global gMoney 
        while True:
            money = random.randint(100, 1000)
            gCondition.acquire()
            while gMoney < money:
                print('余额不足，消费者[%s]需要消费%d元，仅剩%d' % (threading.current_thread(), money, gMoney))
                if gTimes >= gTotalTimes:
                    gCondition.release()
                    return
                gCondition.wait()
            gMoney -= money
            print('消费者[%s]消费%d元，剩余%d' % (threading.current_thread(), money, gMoney))
            gCondition.release()
            time.sleep(0.5)
    
def main():
    for x in range(3):
        t1 = Consumer(name='消费者[%s]' % x)
        t1.start()
        
    for x in range(5):
        t2 = Producer(name='生产者[%s]' % x)
        t2.start()
        
if __name__ == '__main__':
    main()

## Queue线程安全的队列

In [ ]:
from queue import Queue

q = Queue(4) # FIFO 先进先出
q.put(1)
q.put(2)
print(q.qsize())
print(q.empty())
print(q.full())
print(q.get())

In [ ]:
from queue import Queue
import time
import threading

def set_value(q):
    index = 0
    while True:
        q.put(index)
        index += 1
        time.sleep(2)
        
def get_value(q):
    while True:
        print(q.get())
        
def main():
    q = Queue(4)
    t1 = threading.Thread(target=set_value, args=(q,))
    t2 = threading.Thread(target=get_value, args=(q,))
    
    t1.start()
    t2.start()
    
if __name__ == '__main__':
    main()